In [381]:
import math
from pprint import pprint
from math import log, factorial, sqrt, comb
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
    permutations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter, xor, and_, or_, indexOf
from bisect import bisect_right, bisect_left
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import sets
import matrix
import mathematics
import sequences
from importlib import reload

for m in (
    arrays,
    trees,
    lists,
    graphs,
    search,
    strings,
    sets,
    sequences,
    matrix,
    mathematics,
):
    reload(m)

import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
import sets
import matrix as mt
import mathematics as m
from sequences import find_if

print()

In [521]:
def string_with_all_substrings(n: int, k: int) -> str:
    "Return a string that contains all strings of length `n` from the `k` alphabet."
    if k <= 0 or n <= 0:
        return ""
    if k == 1:
        return "0" * n
    digits = "0123456789"[1:k] + "0"
    if n == 1:
        return digits
    o = "0" * n
    v = set([o])
    all = k**n
    while len(v) < all:
        p = o[1 - n :]
        for d in digits:
            s = p + d
            if s not in v:
                v.add(s)
                o += d
                break
    return o

In [524]:
string_with_all_substrings(2, 2)

'00110'

In [502]:
string_with_all_substrings(2, 3)

'0011210220'

In [503]:
string_with_all_substrings(1, 10)

'1234567890'

In [487]:
"A" * -1

''

In [523]:
"a".append("b")

AttributeError: 'str' object has no attribute 'append'